## Imports

In [ ]:
import re
import math
import sqlite3
import requests
import numpy    as np
import pandas   as pd

from datetime   import datetime
from sqlalchemy import create_engine
from bs4        import BeautifulSoup

## Data Collection

In [ ]:
# parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request to URL
page = requests.get( url, headers=headers )

# Beautiful soup object
soup = BeautifulSoup( page.text, 'html.parser' )

# ==================== pagination ======================================
total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get('data-total')
page_number = math.ceil( int( total_item ) / 36)
url02 = url + '?page-size=' + str( int( page_number*36 ))
page = requests.get( url02, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

# ======================= Product Data =======================
products = soup.find( 'ul', class_='products-listing small' ) 
product_list = products.find_all( 'article', class_='hm-product-item')

# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]

# create data frame
data = pd.DataFrame( [product_id, product_name, product_price, product_category] ).T
data.columns = ['product_id', 'product_name', 'price', 'product_category']

## Data Collection by Product

In [ ]:
# empty dataframe
df_compositions = pd.DataFrame()

# unique columns for all products
aux = []

df_pattern = pd.DataFrame( columns = ['Art. No.', 'Composition', 'Fit', 'Size'] )
for i in range ( len( data ) ):

    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    
    page = requests.get( url, headers=headers )

    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )

# ==================== color name ==================================
    product_list = soup.find_all( 'a', class_='filter-option miniature active') + soup.find_all( 'a', class_='filter-option miniature')
    color_name = [p.get('data-color') for p in product_list]

    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]

    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range( len( df_color ) ):
        # API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'

        page = requests.get( url, headers=headers )

        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )
        
        # ====================== Product Name =================================
        product_name = soup.find_all( 'h1', class_='primary product-item-headline')
        product_name = product_name[0].get_text()
        
        # ====================== Product Price =================================
        product_price = soup.find_all( 'div', class_='primary-row product-item-price')
        product_price = re.findall('\d+\.?\d+', product_price[0].get_text())[0]
        
        
        # ==================== composition ==================================
        product_composition_list = soup.find_all( 'div', class_='details-attributes-list-item' )
        product_composition =[list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # rename dataframe
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        # remove pocket lining, shell, pocket and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Pocket lining: ', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Shell: ', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Lining: ', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Pocket: ', '', regex=True )

        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition] )

        # rename columns
        df_composition = df_composition[['Art. No.', 'Composition', 'Fit', 'Size']]
        df_composition.columns = ['product_id', 'composition', 'fit', 'size']
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price

        # keep new columns if it show up
        aux = aux + df_composition.columns.tolist()

        # merge data color + composition
        df_composition = pd.merge( df_composition, df_color, how='left', on='product_id')
        
        df_composition = df_composition.drop_duplicates()
        
        # all products
        df_compositions = pd.concat( [df_compositions, df_composition], axis=0 )
        
    
# generate style id + color id
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:])

# scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S')

## Data Cleaning

In [ ]:
# product id
df_data = df_compositions.dropna( subset=['product_id'])

# product_name
df_data['product_name'] = df_data['product_name'].str.replace('\n', '')
df_data['product_name'] = df_data['product_name'].str.replace('\t', '')
df_data['product_name'] = df_data['product_name'].str.replace('  ', '')
df_data['product_name'] = df_data['product_name'].str.replace(' ', '_').str.lower()

# product_price
df_data['product_price'] = df_data['product_price'].astype( float )

# color_name
df_data['color_name'] = df_data['color_name'].str.replace( ' ', '_' ).str.lower ()

# fit
df_data['fit'] = df_data['fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x)

# size leg
df_data['size_leg'] = df_data['size'].apply( lambda x: re.search( '(\d{2}\.\d) cm', x ).group(1) if pd.notnull(x) else x)

# size number
df_data['size_number'] = df_data['size'].apply( lambda x: re.search('Size (.+)', x).group(1).replace( ')', '') if pd.notnull(x) else x)

# ============================ composition ===========================================
# break composition by comma
df1 = df_data['composition'].str.split(',', expand=True).reset_index(drop=True)

# coton | polyester | spandex
df_ref = pd.DataFrame(index=np.arange(len(df_data)), columns=['cotton', 'polyester', 'spandex'])

# ------------------- coton -------------------
df_cotton_0 = df1.loc[df1[0].str.contains( 'Cotton', na=True), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains( 'Cotton', na=True), 1]
df_cotton_1.name = 'cotton'

# combine
df_cotton = df_cotton_0.combine_first( df_cotton_1)

df_ref = pd.concat( [df_ref, df_cotton], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]

# ------------------- polyester -------------------
df_polyester_0 = df1.loc[df1[0].str.contains( 'Polyester', na=True), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains( 'Polyester', na=True), 1]
df_polyester_1.name = 'polyester'

# combine
df_polyester = df_polyester_0.combine_first( df_polyester_1 )

df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]

# ------------------- spandex -------------------
df_spandex_1 = df1.loc[df1[1].str.contains( 'Spandex', na=True), 1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains( 'Spandex', na=True), 2]
df_spandex_2.name = 'spandex'

df_spandex_3 = df1.loc[df1[3].str.contains( 'Spandex', na=True), 3]
df_spandex_3.name = 'spandex'

# combine spandex from both columns 1 and 2
df_spandex_c2 = df_spandex_1.combine_first( df_spandex_2)
df_spandex = df_spandex_c2.combine_first( df_spandex_3)

df_ref = pd.concat( [df_ref, df_spandex], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]

# join of combine with product_id
df_aux = pd.concat( [df_data['product_id'].reset_index(drop=True),df_ref], axis=1)

# format composition df_data
df_aux['cotton'] = df_aux['cotton'].apply( lambda x: int( re.search('\d+', x).group(0) ) / 100 if pd.notnull( x ) else x)
df_aux['polyester'] = df_aux['polyester'].apply( lambda x: int( re.search('\d+', x).group(0) ) / 100 if pd.notnull( x ) else x)
df_aux['spandex'] = df_aux['spandex'].apply( lambda x: int( re.search('\d+', x).group(0) ) / 100 if pd.notnull( x ) else x)

# final join
df_aux = df_aux.groupby( 'product_id' ).max().reset_index().fillna( 0 )

df_data = pd.merge( df_data, df_aux, on='product_id', how='left' )

# Drop columns
df_data = df_data.drop( columns=['size', 'composition'], axis=1 )

# Drop duplicates
df_data = df_data.drop_duplicates()

In [ ]:
df_data.head()

# Data Insert

In [ ]:
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'fit',
    'product_price',
    'size_number',
    'size_leg',
    'cotton',
    'polyester',
    'spandex',
    'scrapy_datetime'
]]

In [ ]:
query_showroom_schema = """
    CREATE TABLE vitrine(
    product_id         TEXT,
    style_id           TEXT,
    color_id           TEXT,
    product_name       TEXT,
    color_name         TEXT,
    fit                TEXT,
    product_price      REAL,
    size_number        TEXT,
    size_leg           TEXT,
    cotton             REAL,
    polyester          REAL,
    spandex            REAL,
    scrapy_datetime    TEXT
    )
"""



In [ ]:
# create table
conn = sqlite3.connect( 'database_hm.sqlite' )
cursor = conn.execute( query_showroom_schema )
conn.commit() 

In [ ]:
# create database connection
conn = create_engine( 'sqlite:///database_hm.sqlite', echo=False )

# data insert
data_insert.to_sql( 'vitrine', con=conn, if_exists='append', index=False )